# Homework 1: Monty Hall 


# Import 


In [1]:
using Random #for the "Shuffle!" function 
using StatsBase # for the "sample" function 


# Settings 

In [31]:
number_doors = 3 
num_simulation = 1000; 


# mutable structure 


create a mutable structure Door to represent a door with 2 attributes: 
    #whether or not it has been opened by the game show host 
    #what is behinf the door (goat or car)
    #it needs to be mutable so that the game show host can open it 

In [4]:
behind_the_door = Set{Symbol}([:goat, :car])

mutable struct Door 
    open::Bool 
    contains::Symbol 
    
  
end 
    

# Functions

# pick_a_door_at_random()

returns an interger (1, 2, 3) at random, representing the door that the contestant chose.

In [5]:

function pick_a_door_at_random() #randomly picks a value between 1 & 3 to signify which door the contestant has chosen 
    first_door_id_pick = (rand(1:number_doors))
    return first_door_id_pick
end

first_door_id_pick = pick_a_door_at_random()

3

# set_up_door()

set up a function that constructs and returns an array of three Doors(Array{Door}) such that 
    #two doors have a goat behind them 
    #one door has a car behind it 
    #which door has the car is chosen at uniform random 
    #all doors are unopened 

In [6]:

function set_up_doors()
    doors = [Door(false, :goat) for _ in 1:number_doors]
    doors[pick_a_door_at_random()].contains = :car
    return doors 
end

doors = set_up_doors()

3-element Array{Door,1}:
 Door(false, :goat)
 Door(false, :goat)
 Door(false, :car) 

# open_a_door(doors::Array{Door}, first_door_id_pick::Int)


    #the door that the contestant picked is not opened 
    #the door with the car is not opened 
    #if two doors are viable to open, pick one at uniform random 

In [7]:

function open_a_door!(doors::Array{Door}, first_door_id_pick::Int64)
    valid_doors = Array(1:number_doors)
    deleteat!(valid_doors, valid_doors .== first_door_id_pick)
    deleteat!(valid_doors,  [doors[i].contains == :car for i in valid_doors])
        if length(valid_doors) == 1
            doors[valid_doors[1]].open = true
            return doors[valid_doors[1]]
        else
            which_door = rand(valid_doors)
            doors[which_door].open = true
            return which_door
        end
    
end

open_a_door!(doors, first_door_id_pick)
doors

3-element Array{Door,1}:
 Door(false, :goat)
 Door(true, :goat) 
 Door(false, :car) 


# switch_door_pick()

returns the index in {1, 2, 3} of the door that the contestant must pick if he/she switches from his/her initial door choice to the other upopened door. It is assumed in this function that one of the Doors passed has been opened by the game host and has a goat behind it. 

In [8]:

function switch_door_pick(doors::Array{Door}, first_door_id_pick::Int)
    switch_door = Array(1:number_doors)
    deleteat!(switch_door, [d.open for d in doors])
    deleteat!(switch_door, switch_door .== first_door_id_pick)
    return switch_door
end

sd = switch_door_pick(doors, first_door_id_pick)

1-element Array{Int64,1}:
 1

# contestant_wins_car(doors::array{Door}, door_id_pick::Int)

returnns true if the contestant picked the door with the car behind it and false otherwise 

In [9]:
function contestant_wins_car(doors::Array{Door}, door_id_pick)
    if doors[first_door_id_pick].contains == :car 
        return true 
    else
        return false  
    end 
end 

n = contestant_wins_car(doors, first_door_id_pick)


true

# simulate_Monte_Hall(switch::Bool; verbose::Bool=true)

Uses all above functions to simulate the Monte Hall game.
    #returns true if the contestant wins and false otherwise 
    #switch argument specifies whether the contestant uses the strategy to switch doors.
    #verbose arguement specifies whether the function (for debugging and illustration) print off details of the outcome
        #what is behind each door 
        #what door the contestant picked 
        #what door the game show host opened to reveal goat
        #(if applicable) the choice of the door after the contestant switches 
        #if contestant won or lost

In [21]:
do_they_win = Dict{Bool, String}(true => "The Contestant Wins!", false => "The Contestant Loses") 

switch = true


function simulate_Monte_Hall(switch::Bool, verbose::Bool=true)
    doors = set_up_doors()
    first_door_id_pick = pick_a_door_at_random()
    host_pick = open_a_door!(doors, first_door_id_pick)
    
    if switch 
        switch_pick = switch_door_pick(doors, first_door_id_pick)
        outcome = contestant_wins_car(doors, switch_pick)
    else 
        outcome = contestant_wins_car(doors, first_door_id_pick)
    
    end
    
    if verbose 
        for (i, d) in enumerate(doors)
                println("Behind door #", i, " is a ", d.contains)
        end
        println("Contestant picked Door #", first_door_id_pick)
        println("The game show host picked Door #", host_pick)
        switch ? 
            println("Contestant switched to Door #", switch_pick[1]) : print("Contestant keeps Door #", first_door_id_pick )
        println(do_they_win[outcome])
    end
        
    return outcome 
end
   
simulate_Monte_Hall(switch, verbose)

Behind door #1 is a goat
Behind door #2 is a car
Behind door #3 is a goat
Contestant picked Door #2
The game show host picked Door #3
Contestant switched to Door #1
The Contestant Loses


false

# Simulation 

Run 1000 simulations of the Monte Hall problem under the two stategies. Print off how many times the contestant won under both of the strategies. 

In [33]:
strategies = Dict{Bool, String}(
    true => "Switch Doors",
    false => "Sticking with the original door")

wins = Dict{Bool, Int}(
    true => 0, 
    false => 0)

number_switches = length(strategies)
    for s in 1:num_simulation
        for switch in keys(wins) 
            wins[switch] += simulate_Monte_Hall(switch, false)
        end
    end

print("In this simulation there were ")
for (i, switch) in enumerate(keys(wins))
    num_wins = wins[switch]
    percent = 100* Float64(num_wins)/Float64(num_simulation)
    strategy = strategies[switch]
    print(num_wins, " wins ", percent, "% from ", strategy)
    (i < num_switches) ? print("and") : print(".")
    
end
    

In this simulation there were 325 wins 32.5% from Sticking with the original door

UndefVarError: UndefVarError: num_switches not defined